# Today's topic: How to set your configs right: Max Partition Bytes

# 0. Set-Ups

General hints for this notebook:
- Spark UI usually accesible by http://localhost:4040/ or http://localhost:4041/
- Deep dive Spark UI happens in later episodes
- sc.setJobDescription("Description") replaces the Job Description of an action in the Spark UI with your own
- sdf.rdd.getNumPartitions() returns the number partitions of the current Spark DataFrame
- sdf.write.format("noop").mode("overwrite").save() is a good way to analyze and initiate actions for transformations without side effects during an actual write

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import gresearch.spark.parquet
import math
import time
import pandas as pd
import os
import sys


In [3]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [4]:
spark = SparkSession \
    .builder \
    .appName("Data with Nikk the Greek Spark Session") \
    .master("local[4]") \
    .enableHiveSupport() \
    .config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.5") \
    .getOrCreate()

"""
Reference gresearch:
- Parquet files analysis: https://www.gresearch.com/blog/article/parquet-files-know-your-scaling-limits/
- GitHub Spark extension: https://github.com/G-Research/spark-extension
- Parquet methods: https://github.com/G-Research/spark-extension/tree/master/python/gresearch/spark/parquet
"""
sc = spark.sparkContext

In [5]:
#Turning off AQE as it generates more jobs which might be confusing for this scenario here. 
spark.conf.set("spark.sql.adaptive.enabled", "false")
#to not cache datafrimes... this may not create repeatable results
spark.conf.set("spark.databricks.io.cache.enabled", "false")

In [6]:
def sdf_generator(num_rows: int, num_partitions: int = None) -> "DataFrame":
    return (
        spark.range(num_rows, numPartitions=num_partitions)
        .withColumn("date", f.current_date())
        .withColumn("timestamp",f.current_timestamp())
        .withColumn("idstring", f.col("id").cast("string"))
        .withColumn("idfirst", f.col("idstring").substr(0,1))
        .withColumn("idlast", f.col("idstring").substr(-1,1))
        )

In [7]:
def rows_per_partition(sdf: "DataFrame") -> None:
    num_rows = sdf.count()
    sdf_part = sdf.withColumn("partition_id", f.spark_partition_id())
    sdf_part_count = sdf_part.groupBy("partition_id").count()
    sdf_part_count = sdf_part_count.withColumn("count_perc", 100*f.col("count")/num_rows)
    sdf_part_count.orderBy("partition_id").show()

In [8]:
def rows_per_partition_col(sdf: "DataFrame", num_rows: int, col: str) -> None:
    sdf_part = sdf.withColumn("partition_id", f.spark_partition_id())
    sdf_part_count = sdf_part.groupBy("partition_id", col).count()
    sdf_part_count = sdf_part_count.withColumn("count_perc", 100*f.col("count")/num_rows)
    sdf_part_count.orderBy("partition_id", col).show()


In [9]:
BASE_DIR = "D:/Spark/Data"

In [10]:
def write_generator(num_rows, num_files):
    sdf = sdf_generator(num_rows, num_files)
    path = f"{BASE_DIR}/{num_files}_files_{num_rows}_rows.parquet"
    sc.setJobDescription(f"Write {num_files} files, {num_rows} rows")
    sdf.write.format("parquet").mode("overwrite").save(path)
    sc.setJobDescription("None")
    print(f"Num partitions written: {sdf.rdd.getNumPartitions()}")
    print(f"Saved Path: {path}")
    return path

In [11]:
def set_configs(maxPartitionsMB = 128, openCostInMB = 4, minPartitions = 4):
    maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
    openCostInBytes = math.ceil(openCostInMB*1024*1024)
    spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")
    spark.conf.set("spark.sql.files.openCostInBytes", str(openCostInBytes)+"b")
    spark.conf.set("spark.sql.files.minPartitionNum", str(minPartitions))
    print(" ")
    print("******** SPARK CONFIGURATIONS ********")
    print(f"MaxPartitionSize {maxPartitionsMB} MB or {maxPartitionsBytes} bytes")
    print(f"OpenCostInBytes {openCostInMB} MB or {openCostInBytes} bytes")
    print(f"Min Partitions: {minPartitions}")

In [12]:
def get_parquet_meta_data(path):
    sdf = (
        spark.read.parquet_metadata(path)
        .select("filename", "blocks", "compressedBytes", "rows")
        .dropDuplicates(["filename"])
        .withColumn("compressedMB", f.round(f.col("compressedBytes")/1024/1024, 1))
        .withColumn("calcNumBlocks", f.col("compressedMB")/128)
    )
    sdf.show(20, truncate=False)
    return sdf

In [13]:
def get_parquet_blocks(path):
    sdf = (
        spark.read.parquet_blocks(path)
        .dropDuplicates(["filename","block"])
        .orderBy("filename", "block")
        .withColumn("blockEnd", f.col("blockStart") + f.col("compressedBytes") - 1)
        .withColumn("blockMiddle", f.col("blockStart") + 0.5 * f.col("compressedBytes"))
        .withColumn("compressedMB", f.round(f.col("compressedBytes")/1024/1024, 1))
        .select("filename", "block", "blockStart", "blockEnd", "blockMiddle", "compressedBytes", "compressedMB", "rows")
    )

    sdf.show(20, truncate=False)

In [14]:
def get_spark_partitions(path):
    sdf = (
        spark.read.parquet_partitions(path)
        .withColumn("compressedMB", f.round(f.col("compressedBytes")/1024/1024, 1))
        .select("partition", "start", "end", "length", "blocks", "compressedBytes", "compressedMB", "rows", "filename")
    )

    sdf.show(20, truncate=False)

In [15]:
def get_parquet_window_length(path):
    sdf = spark.read.parquet_partitions(path)
    val = sdf.select(f.max(sdf["length"]))
    max_length = val.collect()[0][0]
    print(f"Max Parquet window length: {round(max_length/1024/1024, 1)} MB or {max_length} bytes")

In [16]:
def get_parquet_file_size(path):
    sdf = (
        spark.read.parquet_metadata(path)
        .select("filename", "blocks", "compressedBytes", "rows")
        .dropDuplicates(["filename"])
    )
    sum = sdf.select(f.sum(sdf["compressedBytes"]))
    size = sum.collect()[0][0]
    return size

In [17]:
def bytes_rows_per_partition(path):
    sdf = (
        spark.read.parquet_partitions(path)
        .groupBy("partition").agg(f.sum("compressedBytes"), f.sum("rows"), f.count("partition"))
        .withColumnRenamed("sum(compressedBytes)", "compressedBytes")
        .withColumnRenamed("sum(rows)", "rows")
        .withColumnRenamed("count(partition)", "numFiles")
        .withColumn("compressedMB", f.round(f.col("compressedBytes")/1024/1024, 1))
        .select("partition", "numFiles", "compressedBytes","compressedMB","rows")
        .orderBy("partition")
    )
    sdf.show(20)
    return sdf

In [18]:
def file_analysis(path, num_files):
    file_size = get_parquet_file_size(path)
    avg_file_size = file_size/num_files
    print(" ")
    print("******** FILE SIZE ANALYSIS ********")
    print(f"File Size: {round(file_size/1024/1024, 1)} MB or {file_size} bytes")
    print(f"Num files: {num_files}")
    print(f"Avg file Size: {round(avg_file_size/1024/1024, 1)} MB or {avg_file_size} bytes")
    


In [19]:
def row_count_analysis(num_files, num_rows):
    print(" ")
    print("******** ROW COUNT ANALYSIS ********")    
    print(f"Num files written: {num_files}")
    print(f"Num rows written: {num_rows}")
    print(f"Num rows per file: {int(num_rows/num_files)}")

In [20]:
def get_actual_num_partitions(path):
    sdf = spark.read.parquet(path)
    print(" ")
    print("******** ACTUAL RESULTS ********")   
    print(f"ActualNumPartitions: {sdf.rdd.getNumPartitions()}")
    return sdf.rdd.getNumPartitions()


In [21]:
def noop_write(path, desc = ""):
    sdf = spark.read.parquet(path)
    sc.setJobDescription(f"WRITE {desc}")
    start_time = time.time()
    sdf.write.format("noop").mode("overwrite").save()
    end_time = time.time()
    sc.setJobDescription("None")
    duration = round(end_time - start_time, 2)
    print(f"Duration: {duration} sec")
    return duration

In [22]:
def maxSplitBytes(file_size, num_files):
    """
    Reference to code: 
    - Stackoverflow: https://stackoverflow.com/questions/70985235/what-is-opencostinbytes
    - GitHub: https://github.com/apache/spark/blob/v3.3.1/sql/core/src/main/scala/org/apache/spark/sql/execution/datasources/FilePartition.scala#L86-L97
    """
    maxPartitionBytes = int(spark.conf.get("spark.sql.files.maxPartitionBytes")[:-1])
    openCostInBytes = int(spark.conf.get("spark.sql.files.openCostInBytes")[:-1])
    minPartitionNum = int(spark.conf.get("spark.sql.files.minPartitionNum"))
    
    file_size_padded = file_size + num_files * openCostInBytes
    size_per_core_padded = file_size_padded / minPartitionNum
    max_partition_size = int(min(maxPartitionBytes, max(openCostInBytes, size_per_core_padded)))
    no_partitions_padded = file_size_padded/max_partition_size

    print(" ")
    print("******** ADVANCED ALGORITHM TO ESTIMATE Partition SIZE AND NO PARTITIONS ********")
    print(f"Padded File Size: {round(file_size_padded/1024/1024, 1)} MB or {file_size_padded} bytes")
    print(f"SizePerCorePadded: {round(size_per_core_padded/1024/1024, 1)} MB or {size_per_core_padded} bytes")
    print(f"MaxPartionsize: {round(max_partition_size/1024/1024, 1)} MB or {max_partition_size} bytes")
    print(f"EstimatedPartitionsAvg: {math.ceil(no_partitions_padded)}, unrounded: {no_partitions_padded}")
    return max_partition_size, size_per_core_padded

In [23]:
def get_files_as_list(parquet_meta_data_sdf):
    return list(parquet_meta_data_sdf.select("compressedBytes").toPandas()["compressedBytes"])

In [24]:
def split_files(files_list, max_split_bytes):
    """
    Reference to code in GitHub: https://github.com/apache/spark/blob/master/sql/core/src/main/scala/org/apache/spark/sql/execution/PartitionedFileUtil.scala#L45
    """
    result_list = []
    for file_size in files_list:
        remaining = file_size - max_split_bytes
        f = {
            "start": 0,
            "length": min(max_split_bytes, file_size),
            "file_size": file_size
        }
        result_list.append(f)
        while remaining > 0:
            if remaining > max_split_bytes:
                f = {
                    "start": file_size - remaining,
                    "length": max_split_bytes,
                    "file_size": 0
                }
                result_list.append(f)
            else:
                f = {
                    "start": file_size - remaining,
                    "length": remaining,
                    "file_size": 0
                }
                result_list.append(f)  
            remaining = remaining - max_split_bytes
    return sorted(result_list, key=lambda d: d['length'], reverse=True) 


In [25]:
def getFilePartitions(splitted_files_list, max_split_bytes):
    """
    Reference to code in GitHub: https://github.com/apache/spark/blob/v3.3.1/sql/core/src/main/scala/org/apache/spark/sql/execution/datasources/FilePartition.scala
    """
    openCostInBytes = int(spark.conf.get("spark.sql.files.openCostInBytes")[:-1])
    partitions = []
    current_files = []
    current_size = 0

    def close_partition():
        if current_files:
            partition_details = {
                "files": current_files.copy(),
                "num_files": len(current_files),
            }
        else:
            partition_details = {}
        partitions.append(partition_details)
        current_files.clear()

    for file in splitted_files_list:
        if current_size + file["length"] > max_split_bytes:
            close_partition()
            current_size = 0
        current_size += file["length"] + openCostInBytes
        current_files.append(file)
    close_partition()
    print(f"Number calculated Partitions: {len(partitions)}")
    return partitions


In [26]:
def file_partitions_analysis(file_partitions):
    pdf = pd.DataFrame(file_partitions)
    pdf["partition"] = pdf.index
    sdf_partitions = spark.createDataFrame(pdf)
    sdf_partitions = sdf_partitions.withColumn("files", f.explode(sdf_partitions.files))
    sdf_partitions = (sdf_partitions
                    .withColumn("start", f.col("files").start)
                    .withColumn("length", f.col("files").length)
                    .withColumn("file_size", f.col("files").file_size)
                    .drop("files", "num_files")
                    .withColumn("virt_num_files", f.lit(1))
                    .withColumn("real_num_files", f.when(f.col("file_size") > 0, f.lit(1)).otherwise(f.lit(0)))
                    .select("partition", "start", "length", "file_size", "real_num_files", "virt_num_files")
    )
    #sdf_partitions.show()
    sdf_agg = sdf_partitions.groupBy("partition").agg(f.sum("file_size")).withColumnRenamed("sum(file_size)", "file_size")
    sdf_agg = sdf_agg.orderBy("partition")
    sdf_agg = sdf_agg.withColumn("file_size_mb", f.col("file_size")/1024/1024)
    #sdf_agg.show()
    max_size = sdf_agg.select(f.max(sdf_agg["file_size"])).collect()[0][0]
    min_size = sdf_agg.select(f.min(sdf_agg["file_size"])).collect()[0][0]
    median_size = sdf_agg.select(f.median(sdf_agg["file_size"])).collect()[0][0]
    sdf_empty = sdf_agg.filter(f.col("file_size") == 0)
    empty_partitions = sdf_empty.count()
    return {
        "max_size": max_size,
        "min_size": min_size,
        "median_size": median_size,
        "num_partitions": len(file_partitions),
        "empty_partition": empty_partitions
    }
    

# 1 - MaxPartitionBytes
  - Influences the size of a partition  
  - based on the config"spark.sql.files.maxPartitionBytes"
  - defaults to 128 MB 
  - you can influence the initial distribution of the data across your available cores and depending on your available memory


References:
- https://stackoverflow.com/questions/70985235/what-is-opencostinbytes
- https://stackoverflow.com/questions/69034543/number-of-tasks-while-reading-hdfs-in-spark
- https://stackoverflow.com/questions/75924368/skewed-partitions-when-setting-spark-sql-files-maxpartitionbytes
- https://spark.apache.org/docs/latest/sql-performance-tuning.html
- https://www.linkedin.com/pulse/how-initial-number-partitions-determined-pyspark-sugumar-srinivasan#:~:text=Ideally%20partitions%20will%20be%20created,resource%20will%20get%20utilised%20properly

# 2 - Our Dataset

In [26]:
num_files = 35
num_rows = 256000000
#path = write_generator(num_rows, num_files)

In [26]:
path = "D:/Spark/Data/35_files_256000000_rows.parquet"
num_files = 35
num_rows = 256000000
sdf_meta_data = get_parquet_meta_data(path)
file_analysis(path, num_files)
row_count_analysis(num_files, num_rows)
file_size = get_parquet_file_size(path)
file_size_list = get_files_as_list(sdf_meta_data)


+-----------------------------------------------------------------------------------------------------------------------+------+---------------+-------+------------+-------------+
|filename                                                                                                               |blocks|compressedBytes|rows   |compressedMB|calcNumBlocks|
+-----------------------------------------------------------------------------------------------------------------------+------+---------------+-------+------------+-------------+
|file:/D:/Spark/Data/35_files_256000000_rows.parquet/part-00017-f5a93bc6-81eb-49be-828b-24daa08c7f01-c000.snappy.parquet|1     |64853388       |7314285|61.8        |0.4828125    |
|file:/D:/Spark/Data/35_files_256000000_rows.parquet/part-00032-f5a93bc6-81eb-49be-828b-24daa08c7f01-c000.snappy.parquet|1     |64813324       |7314286|61.8        |0.4828125    |
|file:/D:/Spark/Data/35_files_256000000_rows.parquet/part-00030-f5a93bc6-81eb-49be-828b-24daa08c7f01

# 3 - Experiment dry run

In [63]:
results = []
results_dict = {}

for maxMB in [550]:
    openCostMB = 4
    minPartitions = 4
    set_configs(maxPartitionsMB=maxMB, openCostInMB=openCostMB, minPartitions=minPartitions)
    max_split_bytes, size_per_core_padded = maxSplitBytes(file_size, num_files)
    splitted_files = split_files(file_size_list, max_split_bytes)
    file_partitions = getFilePartitions(splitted_files, max_split_bytes)
    results_dict = file_partitions_analysis(file_partitions)
    results_dict["maxPartitionMB"] = maxMB
    results_dict["openCosts"] = openCostMB
    results_dict["size_per_core_padded"] = round(size_per_core_padded/1024/1024,1)
    results_dict["maxSplitBytes"] = round(max_split_bytes/1024/1024,1)
    results_dict["actualNumberPartitions"] = get_actual_num_partitions(path)
    results_dict["ExecutionTime"] = noop_write(path, str(maxMB))
    bytes_rows_per_partition(path)
    results.append(results_dict)

 
******** SPARK CONFIGURATIONS ********
MaxPartitionSize 550 MB or 576716800 bytes
OpenCostInBytes 4 MB or 4194304 bytes
Min Partitions: 4
 
******** ADVANCED ALGORITHM TO ESTIMATE Partition SIZE AND NO PARTITIONS ********
Padded File Size: 2269.4 MB or 2379647364 bytes
SizePerCorePadded: 567.4 MB or 594911841.0 bytes
MaxPartionsize: 550.0 MB or 576716800 bytes
EstimatedPartitionsAvg: 5, unrounded: 4.126197405728427
Number calculated Partitions: 5
 
******** ACTUAL RESULTS ********
ActualNumPartitions: 5
Duration: 20.96 sec
+---------+--------+---------------+------------+--------+
|partition|numFiles|compressedBytes|compressedMB|    rows|
+---------+--------+---------------+------------+--------+
|        0|       8|      522846371|       498.6|58514285|
|        1|       8|      519680098|       495.6|58514287|
|        2|       8|      511373273|       487.7|58514285|
|        3|       8|      494281964|       471.4|58514285|
|        4|       3|      184665018|       176.1|2194285

In [29]:
pdf = pd.DataFrame(results)
sdf_results = spark.createDataFrame(pdf)
sdf_results = sdf_results.withColumn("min_size", f.round(f.col("min_size")/1024/1024,1))
sdf_results = sdf_results.withColumn("median_size", f.round(f.col("median_size")/1024/1024,1))
sdf_results = sdf_results.withColumn("max_size", f.round(f.col("max_size")/1024/1024,1))
sdf_results = sdf_results.select("maxPartitionMB", "openCosts", "size_per_core_padded", "maxSplitBytes", "num_partitions", "actualNumberPartitions", "empty_partition", "min_size", "median_size", "max_size", "ExecutionTime")
sdf_results.show()

+--------------+---------+--------------------+-------------+--------------+----------------------+---------------+--------+-----------+--------+-------------+
|maxPartitionMB|openCosts|size_per_core_padded|maxSplitBytes|num_partitions|actualNumberPartitions|empty_partition|min_size|median_size|max_size|ExecutionTime|
+--------------+---------+--------------------+-------------+--------------+----------------------+---------------+--------+-----------+--------+-------------+
|           256|        4|               567.4|        256.0|            11|                    11|              0|   117.3|      186.4|   237.7|        23.13|
+--------------+---------+--------------------+-------------+--------------+----------------------+---------------+--------+-----------+--------+-------------+



# 4 - Experiment
- We have a dataset of 2129.4 MB with 35 files each 60.8 MB
- We are looking on the data distribution, no partitions and empty partitions of each dataset for a MaxPartition bytes value between 5 and 1024 MB for this Dataset
- We also compare the performance within the Spark UI  
- We choose a good number of partitions based on no empty partitions, number partitions devidable by 4 (num cores) and a good data distribution
- Here 12 or 16 might be good choices which don't have too big, partitions, devidable by num cores and good data distribution, 8 partitions for 316 MB might also be accaptable

In [164]:
results = []
results_dict = {}

path = "D:/Spark/Data/35_files_256000000_rows.parquet"
num_files = 35
num_rows = 256000000
sdf_meta_data = get_parquet_meta_data(path)
file_analysis(path, num_files)
row_count_analysis(num_files, num_rows)
file_size = get_parquet_file_size(path)
file_size_list = get_files_as_list(sdf_meta_data)

for maxMB in range(5,1025):
    openCostMB = 4
    minPartitions = 4
    set_configs(maxPartitionsMB=maxMB, openCostInMB=openCostMB, minPartitions=minPartitions)
    max_split_bytes, size_per_core_padded = maxSplitBytes(file_size, num_files)
    splitted_files = split_files(file_size_list, max_split_bytes)
    file_partitions = getFilePartitions(splitted_files, max_split_bytes)
    results_dict = file_partitions_analysis(file_partitions)
    results_dict["maxPartitionMB"] = maxMB
    results_dict["openCosts"] = openCostMB
    results_dict["size_per_core_padded"] = round(size_per_core_padded/1024/1024,1)
    results_dict["maxSplitBytes"] = round(max_split_bytes/1024/1024,1)
    results_dict["actualNumberPartitions"] = get_actual_num_partitions(path)
    results_dict["ExecutionTime"] = noop_write(path, str(maxMB))
    #bytes_rows_per_partition(path)
    results.append(results_dict)

+-----------------------------------------------------------------------------------------------------------------------+------+---------------+-------+------------+-------------+
|filename                                                                                                               |blocks|compressedBytes|rows   |compressedMB|calcNumBlocks|
+-----------------------------------------------------------------------------------------------------------------------+------+---------------+-------+------------+-------------+
|file:/D:/Spark/Data/35_files_256000000_rows.parquet/part-00017-f5a93bc6-81eb-49be-828b-24daa08c7f01-c000.snappy.parquet|1     |64853388       |7314285|61.8        |0.4828125    |
|file:/D:/Spark/Data/35_files_256000000_rows.parquet/part-00032-f5a93bc6-81eb-49be-828b-24daa08c7f01-c000.snappy.parquet|1     |64813324       |7314286|61.8        |0.4828125    |
|file:/D:/Spark/Data/35_files_256000000_rows.parquet/part-00030-f5a93bc6-81eb-49be-828b-24daa08c7f01

In [165]:
pdf = pd.DataFrame(results)
sdf_results = spark.createDataFrame(pdf)
sdf_results = sdf_results.withColumn("min_size", f.round(f.col("min_size")/1024/1024,1))
sdf_results = sdf_results.withColumn("median_size", f.round(f.col("median_size")/1024/1024,1))
sdf_results = sdf_results.withColumn("max_size", f.round(f.col("max_size")/1024/1024,1))
sdf_results = sdf_results.select("maxPartitionMB", "openCosts", "size_per_core_padded", "maxSplitBytes", "num_partitions", "actualNumberPartitions", "empty_partition", "min_size", "median_size", "max_size", "ExecutionTime")
sdf_results.show(1025)

+--------------+---------+--------------------+-------------+--------------+----------------------+---------------+--------+-----------+--------+-------------+
|maxPartitionMB|openCosts|size_per_core_padded|maxSplitBytes|num_partitions|actualNumberPartitions|empty_partition|min_size|median_size|max_size|ExecutionTime|
+--------------+---------+--------------------+-------------+--------------+----------------------+---------------+--------+-----------+--------+-------------+
|           525|        4|               567.4|        525.0|             5|                     5|              0|   234.9|      471.9|   495.9|        25.74|
|           526|        4|               567.4|        526.0|             5|                     5|              0|   234.9|      471.9|   495.9|        23.57|
|           527|        4|               567.4|        527.0|             5|                     5|              0|   176.1|      487.7|   498.6|        29.43|
|           528|        4|              

In [166]:
save_path = f"D:/Spark/Data/Results/res.parquet"
sdf_results.write.format("parquet").save(save_path)

In [34]:
load_path = f"D:/Spark/Data/Results/*.parquet"
ddl_schema = "maxPartitionMB bigint, openCosts bigint, size_per_core_padded double, maxSplitBytes double, num_partitions bigint, actualNumberPartitions bigint, empty_partition bigint, min_size double, median_size double, max_size double, ExecutionTime double"
sdf = spark.read.schema(ddl_schema).parquet(load_path)
sdf = sdf.filter(f.col("maxPartitionMB") < 570).orderBy("maxPartitionMB")
sdf = sdf.dropDuplicates(["num_partitions", "min_size", "median_size", "max_size"]).orderBy("maxPartitionMB")
sdf = sdf.filter(f.col("maxPartitionMB") > 63)
sdf.show(1025)

+--------------+---------+--------------------+-------------+--------------+----------------------+---------------+--------+-----------+--------+-------------+
|maxPartitionMB|openCosts|size_per_core_padded|maxSplitBytes|num_partitions|actualNumberPartitions|empty_partition|min_size|median_size|max_size|ExecutionTime|
+--------------+---------+--------------------+-------------+--------------+----------------------+---------------+--------+-----------+--------+-------------+
|           122|        4|               567.4|        122.0|            30|                    31|              0|    59.2|       62.0|   117.9|        22.67|
|           123|        4|               567.4|        123.0|            29|                    29|              0|    58.7|       62.1|   118.6|        21.47|
|           124|        4|               567.4|        124.0|            28|                    28|              0|    61.8|       62.2|   119.3|        21.65|
|           126|        4|              

# 5 - Automatically optimize your partitions
- no partitions dividable by number of cores (here 4)
- no empty partitions
- low data distribution

In [122]:
path = "D:/Spark/Data/35_files_256000000_rows.parquet"

In [128]:
def partition_analyzer(path):
    sdf = (
        spark.read.parquet_partitions(path)
        .select("partition", "rows")
        .groupBy("partition").agg(f.sum("rows"))
        .withColumnRenamed("sum(rows)", "rows")
    )
    return sdf

In [151]:
def number_empty_partitions(sdf):
    return sdf.filter(f.col("rows") == 0).count()

def get_partition_deviation(sdf):
    rows = sdf.select(f.avg("rows"), f.stddev("rows")).collect()
    avg_val = rows[0][0]
    std_val = rows[0][1]
    return std_val/avg_val


In [143]:
sdf = partition_analyzer(path)
sdf.show()
number_empty_partitions = sdf.filter(f.col("rows") == 0).count()
print(number_empty_partitions)
get_partition_deviation(sdf)


+---------+--------+
|partition|    rows|
+---------+--------+
|        1|21942858|
|        6|29257144|
|        3|21942858|
|        5|21942855|
|        9|29257144|
|        4|21942858|
|        8|29257142|
|        7|29257142|
|       10| 7314286|
|        2|21942858|
|        0|21942855|
+---------+--------+

0
23272727.272727273
6391682.907467486


0.27464262493024355

In [205]:
maxMB_start = 50
maxMB_range = 500
num_cores = spark.sparkContext.defaultParallelism
max_deviation = 0.1

set_configs(maxPartitionsMB=128, openCostInMB=4, minPartitions=num_cores)

for maxMB in range(maxMB_start, maxMB_range):
    maxPartitionsBytes = math.ceil(maxMB*1024*1024)
    spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")
    num_partitions = spark.read.parquet(path).rdd.getNumPartitions()
    if num_partitions % num_cores == 0:
        sdf = partition_analyzer(path).cache()
        num_empty = number_empty_partitions(sdf)
        if num_empty == 0:
            dev = get_partition_deviation(sdf)
            if dev <= max_deviation:
                print(f"FOUND: {num_partitions} for MaxPartitionBytes {maxMB} with {num_empty} empty partitions and deviation of {dev}")
            else:
                print(f"SKIPPED: {num_partitions} for MaxPartitionBytes {maxMB} with {num_empty} empty partitions and deviation of {dev}")
        else:
            #print(f"SKIPPED: {num_partitions} for MaxPartitionBytes {maxMB} with {num_empty} empty partitions")
            pass
    else:
        #print(f"SKIPPING: {num_partitions} for MaxPartitionBytes {maxMB}")
        pass

 
******** SPARK CONFIGURATIONS ********
MaxPartitionSize 128 MB or 134217728 bytes
OpenCostInBytes 4 MB or 4194304 bytes
Min Partitions: 4
SKIPPED: 28 for MaxPartitionBytes 124 with 0 empty partitions and deviation of 0.440958534619162
SKIPPED: 28 for MaxPartitionBytes 125 with 0 empty partitions and deviation of 0.440958534619162
SKIPPED: 28 for MaxPartitionBytes 126 with 0 empty partitions and deviation of 0.4409585346191619
SKIPPED: 28 for MaxPartitionBytes 127 with 0 empty partitions and deviation of 0.4409585346191619
SKIPPED: 16 for MaxPartitionBytes 185 with 0 empty partitions and deviation of 0.20155644229431519
SKIPPED: 16 for MaxPartitionBytes 186 with 0 empty partitions and deviation of 0.2719527967296773
SKIPPED: 16 for MaxPartitionBytes 187 with 0 empty partitions and deviation of 0.2719527967296773
SKIPPED: 16 for MaxPartitionBytes 188 with 0 empty partitions and deviation of 0.2719527967296773
SKIPPED: 16 for MaxPartitionBytes 189 with 0 empty partitions and deviation o

# 6 - Execution time

In [27]:
set_configs(maxPartitionsMB=128, openCostInMB=4, minPartitions=4)

 
******** SPARK CONFIGURATIONS ********
MaxPartitionSize 128 MB or 134217728 bytes
OpenCostInBytes 4 MB or 4194304 bytes
Min Partitions: 4


In [35]:
results = []
path = "D:/Spark/Data/35_files_256000000_rows.parquet"
for maxMB in [185]: #[10, 20, 63, 124, 128, 186, 195, 196, 260, 311, 316, 326, 392, 563]:
    maxPartitionsBytes = math.ceil(maxMB*1024*1024)
    spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")
    num_partitions = spark.read.parquet(path).rdd.getNumPartitions()
    print(f"MaxBytes {maxMB}")
    print(f"NumPartitions: {num_partitions}")
    noop_write(path, str(maxMB))

MaxBytes 185
NumPartitions: 16
Duration: 29.06 sec
